In [ ]:
import cx_Oracle
from os import getenv

import numpy as np
import pandas as pd

import datetime, json
import os
from google.cloud import secretmanager

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import figure
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20,12)

import seaborn as sns

import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv("UID"),
    password=os.getenv("PWD"),
    host = os.getenv("HOST"),
    service = os.getenv("SERVICE"),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data(sql):
    user = oracle_secrets['user'] + '[DVH_FAM_PP]' #f"[{schema}]"
    dsn_tns = cx_Oracle.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    try:
        conn = cx_Oracle.connect(user = user, password = oracle_secrets['password'], dsn = dsn_tns)
        df = pd.read_sql(sql = sql, con = conn)
        conn.close()
        return df
    except cx_Oracle.Error as error:
        print(error)

In [ ]:
sql = """ 
SELECT A.PERIODE, DIM_TID.DATO, A.UTBET_FOM, A.UTBET_TOM, A.FK_PERSON1_MOTTAKER, A.FK_PERSON1_PLEIETRENGENDE, A.UTFALL, A.OPPGITT_TILSYN
,A.SAKSNUMMER, A.ARBEIDSFORHOLD_TYPE, A.PLEIEBEHOV
,B.GMT_OVERSE_ETABLERT_TILSYN_AARSAK, B.GMT_ETABLERT_TILSYN
,AARSAK.AARSAK
FROM fam_pp_stonad A
JOIN fam_pp_perioder B ON A.FK_PP_PERIODER=B.PK_PP_PERIODER
LEFT OUTER JOIN fam_pp_periode_aarsak AARSAK ON AARSAK.FK_PP_PERIODER=B.PK_PP_PERIODER 
JOIN DT_P.DIM_TID DIM_TID ON DIM_TID.DIM_NIVAA=1 
AND DIM_TID.DAG_I_UKE<6 
AND DIM_TID.GYLDIG_FLAGG=1 
AND A.UTBET_FOM<=DIM_TID.DATO 
AND A.UTBET_TOM>=DIM_TID.DATO
WHERE A.KILDESYSTEM='PP_VEDTAK' AND A.GYLDIG_FLAGG=0 AND A.PERIODE>202200 AND A.PERIODE<202205
and to_char(DIM_TID.DATO,'YYYYMM')=PERIODE"""

df = get_data(sql = sql)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
filt = (df['PERIODE'] == '202201') & (df['FK_PERSON1_MOTTAKER'] == 627466587) #df['DATO'] == '2022-01-03'
df.loc[filt].sort_values(by = 'DATO')[0:10]

In [ ]:
df['AARSAK'].unique()

In [ ]:
#filt = (df['UTBET_FOM'] == '2022-01-03') 
#len(df.loc[filt, 'FK_PERSON1_MOTTAKER'].unique())

In [ ]:
filt = (df['DATO'] == '2022-01-12') & (df['UTFALL'] == 'OPPFYLT')
len(df.loc[filt, 'FK_PERSON1_MOTTAKER'].unique())

In [ ]:
filt = ((df['FK_PERSON1_MOTTAKER'] == 1865745307) & (df['PERIODE'] == '202202'))
df.loc[filt][:4]

In [ ]:
filt = ((df['UTBET_FOM'] <= '20220112') & (df['UTBET_TOM'] >= '20220112'))
hans = df.loc[filt]

In [ ]:
hans

In [ ]:
filt = (hans['UTFALL'] == 'OPPFYLT')
len(hans.loc[filt, 'FK_PERSON1_MOTTAKER'].unique())

In [ ]:
df.isna().sum().iplot(kind = 'bar')

In [ ]:
pd.set_option('display.max_rows', 20)

# Bestilling av stønadsstatistikk i forbindelse med endringer i 9-11

- Trenger totalt antall saker per dag (som er i søknadsperioden).
- Hvilken tidsperiode uttrekket gjelder for
    - Fullt uttrekk for alle data fra K9 
    - Data per dag dataene gjelder for.

### total antall saker per dag

In [ ]:
ant_saker_per_dag = df.groupby('DATO')['SAKSNUMMER'].nunique().reset_index().set_index('DATO').rename(columns = {'SAKSNUMMER': 'ANTALL_SAKER'})
ant_saker_per_dag

In [ ]:
#df[['DATO', 'SAKSNUMMER']]

In [ ]:
ant_saker_per_dag.iplot(kind = 'bar')

- I hvor mange saker er det opplyst om timer med tilsynsordning?
    - Antall saker med tilsynsordning per dag.

### Antall saker med tilsynsordning per dag

In [ ]:
#filt = ((df['GMT_ETABLERT_TILSYN'].notna()) & (df['GMT_ETABLERT_TILSYN'] > 0))
#df.loc[filt].groupby('DATO')['SAKSNUMMER'].nunique().reset_index().set_index('DATO').rename(columns = {'SAKSNUMMER': 'ANTALL_SAKER'})[:20]

In [ ]:
#df.loc[filt].groupby('DATO')['SAKSNUMMER'].nunique().iplot()

### Antall saker med tilsynsordning per dag UTFALL OPPFYLT og ikke OPPFYLT

In [ ]:
#column_order = ['OPPFYLT', 'IKKE_OPPFYLT', 'TOTAL']

#df.loc[filt][['DATO', 'SAKSNUMMER', 'UTFALL']].pivot_table(
#    index = 'DATO', columns = 'UTFALL', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), 
#                    margins = True, margins_name = 'TOTAL').reindex(column_order, axis = 1)

In [ ]:
filt = ((df['GMT_ETABLERT_TILSYN'].notna()) & (df['GMT_ETABLERT_TILSYN'] > 0))

column_order = ['OPPFYLT', 'IKKE_OPPFYLT', 'TOTAL']

tilsynsordning = df.loc[filt][['DATO', 'SAKSNUMMER', 'UTFALL']].pivot_table(
    index = 'DATO', columns = 'UTFALL', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), 
                    margins = True, margins_name = 'TOTAL').reindex(column_order, axis = 1)

In [ ]:
tilsynsordning.columns = [''] * len(tilsynsordning.columns)
tilsynsordning.columns =['Ant_sak_tilsyn_opp', 'Ant_sak_tilsyn_ikke_opp', 'Ant_sak_tilsyn']
tilsynsordning

In [ ]:
tilsynsordning.iplot()

- I hvor mange av sakene med tilsynsordning er det vurdert nattevåk og/eller beredskap?
    - Antall saker med innvilget beredskap per dag.
    - Antall saker med avslått beredskap per dag.
    - Antall saker med innvilget nattevåk per dag.
    - Antall saker med avslått nattevåk per dag.
    - Antall saker med innvilget beredskap & nattevåk per dag.
    - Antall saker med avslått beredskap & nattevåk per dag.

### Antall saker med innvilget beredskap per dag

In [ ]:
#filt = ((df['GMT_ETABLERT_TILSYN'].notna()) & (df['GMT_ETABLERT_TILSYN'] > 0) ) # henter saker med tilsynsordning

#df.loc[filt][['DATO', 'SAKSNUMMER', 'UTFALL', 'GMT_OVERSE_ETABLERT_TILSYN_AARSAK']].pivot_table(
#    index = ['DATO', 'UTFALL'], columns = 'GMT_OVERSE_ETABLERT_TILSYN_AARSAK', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), 
#                    fill_value=0, margins = True).rename_axis(None, axis = 1)[:24]

In [ ]:
filt = ( (df['GMT_ETABLERT_TILSYN'].notna()) & (df['GMT_ETABLERT_TILSYN'] > 0) & (df['GMT_OVERSE_ETABLERT_TILSYN_AARSAK'] != 'FOR_LAVT')) # henter saker med tilsynsordning

tilsynsordning_med_aarsak = df.loc[filt][['DATO', 'SAKSNUMMER', 'UTFALL', 'GMT_OVERSE_ETABLERT_TILSYN_AARSAK']].pivot_table(
    index = ['DATO', 'UTFALL'], columns = 'GMT_OVERSE_ETABLERT_TILSYN_AARSAK', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), 
                    fill_value=0).rename_axis(None, axis = 1).unstack()

In [ ]:
tilsynsordning_med_aarsak.columns = [''] * len(tilsynsordning_med_aarsak.columns)
tilsynsordning_med_aarsak.columns =['BEREDSKAP_IKKE_OPP', 'BEREDSKAP_OPP','NATTEVÅK_IKKE_OPP', 'NATTEVÅK_OPP', 'NATTEVÅK_OG_BEREDSKAP_IKKE_OPP', 'NATTEVÅK_OG_BEREDSKAP_OPP']
tilsynsordning_med_aarsak

In [ ]:
tilsynsordning_med_aarsak.iplot(kind = 'bar')

- I hvor mange av sakene er det unntak fra gradering på grunn av nattevåk og/eller beredskap?
    - Antall saker der nattevåk og/eller beredskap blir brukt som unntak per dag.

- Antall som graderes med årsak per dag:
    - GRADERT_MOT_TILSYN
    - AVKORTET_MOT_INNTEKT

In [ ]:
filt = ((df['AARSAK'] == 'AVKORTET_MOT_INNTEKT') | (df['AARSAK'] == 'GRADERT_MOT_TILSYN'))
gradering_med_aarsak = df.loc[filt, ['DATO', 'AARSAK', 'SAKSNUMMER']].pivot_table(index = 'DATO', columns = 'AARSAK', values = 'AARSAK',
                                                           aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
gradering_med_aarsak.columns = [''] * len(gradering_med_aarsak.columns)
gradering_med_aarsak.columns =['ANT_SAK_AVKORTET_MOT_INNTEKT', 'ANT_SAK_GRADERT_MOT_TILSYN']
gradering_med_aarsak

- Antall avslag med årsak per dag:
    - FOR_LAV_REST_PGA_ETABLERT_TILSYN 
    - FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE
    - FOR_LAV_TAPT_ARBEIDSTID
    - FOR_LAV_REST_PGA_ANDRE_SØKERE

In [ ]:
avslag_aarsak_list = ['FOR_LAV_REST_PGA_ETABLERT_TILSYN', 'FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE', 'FOR_LAV_TAPT_ARBEIDSTID', 'FOR_LAV_REST_PGA_ANDRE_SØKERE']
filt = (df['AARSAK'].isin(avslag_aarsak_list))
avslag_aarsak = df.loc[filt, ['DATO', 'AARSAK', 'SAKSNUMMER']].pivot_table(index = 'DATO', columns = 'AARSAK', values = 'SAKSNUMMER',
                                                          aggfunc = lambda x: len(x.unique()), fill_value = 0)

In [ ]:
avslag_aarsak.columns = [''] * len(avslag_aarsak.columns)
avslag_aarsak.columns =['ANT_SAK_FOR_LAV_REST_PGA_ANDRE_SØKERE', 'ANT_SAK_FOR_LAV_REST_PGA_ETABLERT_TILSYN', 'ANT_SAK_FOR_LAV_REST_PGA_ETABLERT_TILSYN_OG_ANDRE_SØKERE', 'ANT_SAK_FOR_LAV_TAPT_ARBEIDSTID']
avslag_aarsak

### Antall avslag med årsak per dag (alle årsaker)

In [ ]:
filt = (df['UTFALL'] == 'IKKE_OPPFYLT')
df.loc[filt, ['DATO', 'AARSAK', 'SAKSNUMMER']].pivot_table(index = 'DATO', columns = 'AARSAK', values = 'SAKSNUMMER',
                                                          aggfunc = lambda x: len(x.unique()), fill_value = 0)

- Hvor mange har ikke arbeid?
    - Alle aktiviteter bortsett fra selvstendig næringsdrivende, frilanser og/eller arbeidstaker.

In [ ]:
bortsett_liste = ['AT','SN','FL', None]
filt = (df['UTFALL'] == 'OPPFYLT') & (~df['ARBEIDSFORHOLD_TYPE'].isin(bortsett_liste))
ikke_arbeid = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO',values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique())
                                                ,fill_value = 0)

In [ ]:
ikke_arbeid.columns = [''] * len(ikke_arbeid.columns)
ikke_arbeid.columns =['ANT_SAK_UTEN_ARBEID']
ikke_arbeid

In [ ]:
ikke_arbeid.iplot()

- Hvor mange har ikke arbeid og oppgir tilsyn?
    - Antall saker med arbeid og oppgitt tilsyn per dag.

In [ ]:
arbeid_liste = ['AT','SN','FL']
filt = ((df['GMT_ETABLERT_TILSYN'].notna()) & (df['GMT_ETABLERT_TILSYN'] > 0) & (df['UTFALL'] == 'OPPFYLT') & (df['ARBEIDSFORHOLD_TYPE'].isin(arbeid_liste)))
saker_arbeid_tilsyn = df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER',
                                                aggfunc = lambda x: len(x.unique()), fill_value = 0)

In [ ]:
saker_arbeid_tilsyn.columns = [''] * len(saker_arbeid_tilsyn.columns)
saker_arbeid_tilsyn.columns =['ANT_SAK_MED_ARBEID_OG_TILSYN']
saker_arbeid_tilsyn

- Hvor mange saker har flere søkere i samme periode?
    - Antall saker for et gitt antall søkere (1,2,3,4 eller 5) per dag. (ikke så viktig)

- I hvor mange saker er det kun behov for 1 omsorgsperson?
    - Antall saker for en gitt pleiegrad (0%, 100%, eller 200%) per dag.

In [ ]:
filt = (df['UTFALL'] == 'OPPFYLT')
pleiebehov = df.loc[filt,['DATO', 'PLEIEBEHOV', 'SAKSNUMMER']].pivot_table(index = 'DATO',columns = 'PLEIEBEHOV', values = 'SAKSNUMMER',
                                                    aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
pleiebehov.columns = [''] * len(pleiebehov.columns)
pleiebehov.columns =['ANT_SAK_100_PLEIEBEHOV', 'ANT_SAK_200_PLEIEBEHOV']
pleiebehov

- Dette uttrekket er ikke per dag, men totalt over alle periodene til en fagsak:
    - Hvor mange saker graderes/avslås uten årsakene AVKORTET_MOT_INNTEKT og/eller FOR_LAV_TAPT_ARBEIDSTID
    - Hvor mange saker graderes/avslås uten årsaken GRADERT_MOT_TILSYN

In [ ]:
df['AARSAK'].unique()

### Antall saker graderes/avslås uten årsakene AVKORTET_MOT_INNTEKT og/eller FOR_LAV_TAPT_ARBEIDSTID

In [ ]:
bortsett_aarsak = ['AVKORTET_MOT_INNTEKT', 'FOR_LAV_TAPT_ARBEIDSTID']
filt = (~df['AARSAK'].isin(bortsett_aarsak)) 

df.loc[filt][['UTFALL', 'SAKSNUMMER']].pivot_table(
    index = 'UTFALL', values = 'SAKSNUMMER' ,aggfunc = lambda x: len(x.unique()), fill_value=0)

In [ ]:
ant_sak_avslås_uten_bortsett_aarsak

### Antall saker graderes/avslås uten årsaken GRADERT_MOT_TILSYN

In [ ]:
filt = (df['AARSAK'] != 'GRADERT_MOT_TILSYN') 
df.loc[filt,['UTFALL', 'SAKSNUMMER']].pivot_table(index = 'UTFALL', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()), fill_value=0)

- Antall saker med årsak FOR_LAV_TAPT_ARBEIDSTID per dag
- Antallet saker som ikke har minst én søknadsperiode som er lenger enn 5 dager sammenhengende.

### Antall saker med årsak FOR_LAV_TAPT_ARBEIDSTID per dag

In [ ]:
filt = (df['AARSAK'] == 'FOR_LAV_TAPT_ARBEIDSTID')
df.loc[filt, ['DATO', 'SAKSNUMMER']].pivot_table(index = 'DATO', values = 'SAKSNUMMER', aggfunc = lambda x: len(x.unique()), fill_value = 0)

### Antallet saker som ikke har minst én søknadsperiode som er lenger enn 5 dager sammenhengende.